In [ ]:
from src import SantaFeAgent, SantaFeFood, SantaFeWorld
from src import terminals
from grammaticalevolutiontools.evolution import cross_over, mutation

from tqdm import tqdm
import random
import heapq

%matplotlib widget

In [ ]:
# Defines colors for the agents and objects in the WorldAnimation
AGENT_CMAP = {SantaFeAgent: 'red'}
OBJ_CMAP = {SantaFeFood: 'lime'}

In [ ]:
# Here is a simple implementation of a genetic algorithm, for 
# demonstration purposes

world = SantaFeWorld()
best_agent = None
best_score = -1

num_generations = 10
num_agents = 100
num_to_keep = 50
num_rand = 10
num_ticks = 100

chance_to_keep_best_agent = 0.7
chance_of_mutation = 0.7

agents = [SantaFeAgent() for _ in range(num_agents)]
top_agents_heap = []
best_agent = None

for gen in range(num_generations):
    print(f"GENERATION {gen + 1}: ")
    top_agents_heap.clear()

    print('running and scoring agents...')
    for agent in tqdm(agents):
        # reset the world with the current agent and run the simulation
        world.reset_with_agent(agent)
        world.tick(num_ticks)

        # keep track of n agents with highest scores from this generation
        # using a heap (used for creating next generation).
        if len(top_agents_heap) < num_to_keep:
            heapq.heappush(top_agents_heap, (agent.score + 1, agent))
            # (score + 1) ensures each of the top 10 agents has a non-zero
            # chance of being chosen to reproduce.
        elif (agent.score + 1) > top_agents_heap[0][0]:
            heapq.heapreplace(top_agents_heap, (agent.score + 1, agent))

        # track agent with overall best score across all generations
        if agent.score > best_score:
            best_agent = agent
            best_score = best_agent.score

    print('selecting agents...')
    # weight each top agent by its score
    weights, top_agents = zip(*top_agents_heap)

    # add some random new agents to add variation (give weight of 1).
    breeding_pool = list(top_agents) + [SantaFeAgent() for _ in range(num_rand)]
    selection_weights = list(weights) + [1]*num_rand

    # chance of putting the overall best agent back into the breeding pool
    if random.random() < chance_to_keep_best_agent:
        breeding_pool += [best_agent]
        selection_weights += [best_score]

    print('creating new generation...')
    new_agents = []
    while len(new_agents) < num_agents:
        # randomly select two agents, 
        agent1, agent2 = random.choices(
            breeding_pool, 
            weights=selection_weights, 
            k=2
            )
        
        # cross over their programs
        prog1, prog2 = cross_over.cross_over_programs(
            agent1.program, 
            agent2.program
            )
        
        # mutation
        if random.random() < chance_of_mutation:
            mutation.mutate_terminals(
                prog1, 
                num_mutations=2, 
                terminal_types=terminals
                )

        new_agents.append(SantaFeAgent(prog1))
        new_agents.append(SantaFeAgent(prog2))

    agents = new_agents


print(best_score)
print(best_agent.program)

In [ ]:
# Rerun the best agent in the world, this time recording each frame, and
# create an animation
world.reset_with_agent(best_agent, recording_on=True)
world.tick(num_ticks)
anim = world.generate_animation(
    bg_color='black', 
    agent_colors=AGENT_CMAP, 
    obj_colors=OBJ_CMAP,
    arrow_color='blue'
    )

In [ ]:
# This animation can be replayed again and again
anim.play(pause=50)